*Import* Necessory Libraries

In [ ]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import string
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report




import warnings
warnings.filterwarnings('ignore')


Creating Dataframe using our dataset

In [ ]:
df =pd.read_csv('emotion_datset_sample - Sheet1.csv')

In [ ]:
df.head()

,SI No.,Comments,(1)\nHappy,(2)\nSad,(3)\nAnger
0,1,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് 🙏🙏🙏മോ...,0,0,1
1,2,തിയേറ്ററിൽ ആളുകൾ ഉറങ്ങിയാണല്ലോ ആറാട്ട് നെ സ്വീ...,0,0,0
2,3,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്...,1,0,0
3,4,ആർക്കോ വേണ്ടി കോമാളിത്തരം കാണിക്കുന്ന സൂപ്പർ സ...,0,0,0
4,5,Adipoli movie 5/5. Unexpected climax👏,1,0,0


Renaming Columns

In [ ]:
df.rename(columns={'(1)\nHappy': 'Happy', '(2)\nSad': 'Sad', '(3)\nAnger': 'Anger'}, inplace=True)
df.head()

,SI No.,Comments,Happy,Sad,Anger
0,1,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് 🙏🙏🙏മോ...,0,0,1
1,2,തിയേറ്ററിൽ ആളുകൾ ഉറങ്ങിയാണല്ലോ ആറാട്ട് നെ സ്വീ...,0,0,0
2,3,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്...,1,0,0
3,4,ആർക്കോ വേണ്ടി കോമാളിത്തരം കാണിക്കുന്ന സൂപ്പർ സ...,0,0,0
4,5,Adipoli movie 5/5. Unexpected climax👏,1,0,0


In [ ]:
print("count of")
print("happy:",df['Happy'].sum())
print("sad:",df['Sad'].sum())
print("anger:",df['Anger'].sum())

count of
happy: 4
sad: 66
anger: 4


<b>Data Preprocessing

Removing Digits,Emojis,English characters

In [ ]:
def clean_text(text):
    # Remove English characters
    text = re.sub(r'[a-zA-Z]', '', text)
    # Remove digits
    text = re.sub(r'\d', '', text)
    # Remove emojis
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', '', text)
    return text

In [ ]:
df['Comments']=df['Comments'].apply(clean_text)

In [ ]:
df.head()

,SI No.,Comments,Happy,Sad,Anger
0,1,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0,0,1
1,2,തിയേറ്ററിൽ ആളുകൾ ഉറങ്ങിയാണല്ലോ ആറാട്ട് നെ സ്വീ...,0,0,0
2,3,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1,0,0
3,4,ആർക്കോ വേണ്ടി കോമാളിത്തരം കാണിക്കുന്ന സൂപ്പർ സ...,0,0,0
4,5,/.,1,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SI No.    100 non-null    int64 
 1   Comments  100 non-null    object
 2   Happy     100 non-null    int64 
 3   Sad       100 non-null    int64 
 4   Anger     100 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 4.0+ KB


Removing rows of comments without marked emotions(ie,evry emtn mrkd as 0)

In [ ]:
#retrieving emotion columns
columns_to_check = df.columns.difference(['Comments','SI No.'])
columns_to_check

Index(['Anger', 'Happy', 'Sad'], dtype='object')

In [ ]:
#identifiying and masking comments without marked emotions
mask = (df[columns_to_check] == 0).all(axis=1)
mask   #true-> every value is 0

0     False
1      True
2     False
3      True
4     False
      ...  
95    False
96    False
97     True
98     True
99     True
Length: 100, dtype: bool

In [ ]:
#removing columns marked as mask=true
df = df[~mask]
df.head()

,SI No.,Comments,Happy,Sad,Anger
0,1,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0,0,1
2,3,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1,0,0
4,5,/.,1,0,0
5,6,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,1,0,0
6,7,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,1,0,0


Correcting Index

In [ ]:
df = df.drop('SI No.', axis=1)
df = df.reset_index(drop=True)

In [ ]:
df.head()

,Comments,Happy,Sad,Anger
0,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0,0,1
1,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1,0,0
2,/.,1,0,0
3,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,1,0,0
4,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,1,0,0


Adding emotion column and removing others

In [ ]:
df['Emotion'] = ''

In [ ]:
df.head()

,Comments,Happy,Sad,Anger,Emotion
0,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0,0,1,
1,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1,0,0,
2,/.,1,0,0,
3,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,1,0,0,
4,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,1,0,0,


In [ ]:
#assigning values to the column
def determine_emotion(row):
    emotion_columns = ['Happy', 'Sad', 'Anger']

    for emotion in emotion_columns:
        if row[emotion] == 1:
            return emotion

    return ''

In [ ]:
df['Emotion'] = df.apply(determine_emotion, axis=1)

In [ ]:
df.head()

,Comments,Happy,Sad,Anger,Emotion
0,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0,0,1,Anger
1,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1,0,0,Happy
2,/.,1,0,0,Happy
3,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,1,0,0,Happy
4,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,1,0,0,Happy


In [ ]:
emotion_columns = ['Happy', 'Sad', 'Anger']
df = df.drop(emotion_columns, axis=1)

In [ ]:
df.head()

,Comments,Emotion
0,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,Anger
1,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,Happy
2,/.,Happy
3,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,Happy
4,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,Happy


Encoding Labels

In [ ]:
label_encoder = LabelEncoder()
df['Emotion'] = label_encoder.fit_transform(df['Emotion'])

In [ ]:
df.head()

,Comments,Emotion
0,ദൈവത്തെ ഓർത്തു ഇനി സം.. വിധാനം ചെയ്യരുത് മോഹൻല...,0
1,അപ്പോ തുടങ്ങിയല്ലേ നെയ്യാറ്റിൻകര ഗോപന്റെ ആറാട്ട്,1
2,/.,1
3,ഇങ്ങനെ ഉള്ള സിനിമകൾ വരണമെങ്കിൽ പ്രിത്വി തന്നെ ...,1
4,ഒരുപാട് വർഷങ്ങൾക്ക് ശേഷം മനസ്സിൽ സ്പർശിച്ച ഒരു...,1


In [ ]:
df.value_counts('Emotion')

Emotion
2    66
1     4
0     1
dtype: int64

<b>Tokenizing Comments

In [ ]:
def tokenize(comment):
    words = word_tokenize(comment)
    filtered_words = [word for word in words if word.lower() not in string.punctuation]
    return filtered_words

In [ ]:
df['tokenized_comments'] = df['Comments'].apply(tokenize)

In [ ]:
df['tokenized_comments']

0     [ദൈവത്തെ, ഓർത്തു, ഇനി, സം, .., വിധാനം, ചെയ്യരു...
1     [അപ്പോ, തുടങ്ങിയല്ലേ, നെയ്യാറ്റിൻകര, ഗോപന്റെ, ...
2                                                    []
3     [ഇങ്ങനെ, ഉള്ള, സിനിമകൾ, വരണമെങ്കിൽ, പ്രിത്വി, ...
4     [ഒരുപാട്, വർഷങ്ങൾക്ക്, ശേഷം, മനസ്സിൽ, സ്പർശിച്...
                            ...                        
66    [ഒരുപാട്, വേദനകൾ, സഹിച്ച, സഹോദരി, വാർത്തകൾ, കേ...
67    [ഇദ്, കാണുമ്പോൾ, നെഞ്ച്, പിടയുന്നു, ..., കണ്ണ്...
68    [അകാലത്തിൽ, പൊഴിഞ്ഞ, പുഷ്‌പ്പത്തിന്, ബാഷ്പ്പാഞ...
69    [വേദനകളില്ലാത്ത, ലോകത്തേയ്ക്ക്, പോയ, സഹോദരിയ്ക...
70                     [ആദരാഞ്ജലികൾ, ഒരുപാട്, വേദനയോടെ]
Name: tokenized_comments, Length: 71, dtype: object

<b>Testing data and Training data split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['tokenized_comments'], df['Emotion'], test_size=0.2, random_state=42)
X_train

30    [അഭിനയിക്കാനുള്ള, ഒരുപാട്, മോഹങ്ങൾ, ബാക്കി, നി...
59    [അവൾ, നടന്നു, തുടങിയപ്പോ, ആ, അമ്മ, എത്രത്തോളം,...
35      [വാക്കുകൾ, ഇല്ല, പറയാൻ, നല്ല, ആക്ടർ, ആയിരുന്നു]
58    [കുട്ടികളെ, പീഡിപ്പിക്കുന്നവനും, ..., കള്ളന്മാ...
42                          [പാവം, എത്ര, വേദന, സഹിച്ചു]
16    [ലാലേട്ടന്റെ, പെങ്ങളായി, അഭിനയിച്ചു, കണ്ടു, കൊ...
34                        [നന്ദുവിനും, ശരണ്യ, മോൾക്കും]
44    [പാവം, ആ, ചിരിക്ക്‌, മുന്നിൽ, തോറ്റു, പോകും, എ...
28    [തിരിച്ചു, വരുമെന്ന്, കരുതിയിരുന്നു, ആരുടെയും,...
7                  [ആ, അമ്മയുടെ, അവസ്ഥ, എന്തായിരിക്കും]
54       [ഇനിയാർക്കൂ, ഈശ്വരൻ, ഈ, വിധി, നൽകാതിരിക്കട്ടെ]
50    [പാവം, ഒത്തിരി, ഇഷ്ടമായിരുന്നു.സുന്ദരി, കുട്ട്യേ]
45    [ചേർന്ന്, നിൽക്കേണ്ടാവർ, പിൻതിരിഞ്ഞ്, പോകുമ്പോ...
47                   [ദുനിയാവിൽ, ഒന്നും, നിലനിൽക്കില്ല]
19    [ഇത്‌, എല്ലാം, കണ്ടു, സഹിക്കാൻ, കഴിയുന്നില്ല, ...
57    [എന്താ, പറയ്യ, കരച്ചിൽ, വരാ, പോയില്ലെ, ഇനി, എന...
40    [മരണമേതു, രൂപത്തിലെത്തുമെന്ന്, മാനവന്, ഒരു, നാ...
25    [കലാഭവൻ, മണിച്ചേട്ടൻ, പിരിഞ്ഞതിന്, ശേഷം, ഏ

In [ ]:
X_train = [" ".join(tokens) for tokens in X_train]
X_test = [" ".join(tokens) for tokens in X_test]
#X_train was a list of lists where each inner list contains tokenized words

In [ ]:
X_train

['അഭിനയിക്കാനുള്ള ഒരുപാട് മോഹങ്ങൾ ബാക്കി നിർത്തി ഒരു പാട് വേദനയും തിന്ന് വേദന ഇല്ലാത്ത ലോകത്തേക്ക് പോയി വേദനയോടെ നല്ല മനസ്സിന്റെ ഉടമ ആയിരുന്നു ആദരാജ്ഞലികൾ',
 'അവൾ നടന്നു തുടങിയപ്പോ ആ അമ്മ എത്രത്തോളം സന്തോഷം ഉണ്ടായിരുന്നു ആ മുഖത് എല്ലാം വലിയ സങ്കടം താരനായിരുന്നോ',
 'വാക്കുകൾ ഇല്ല പറയാൻ നല്ല ആക്ടർ ആയിരുന്നു',
 'കുട്ടികളെ പീഡിപ്പിക്കുന്നവനും ... കള്ളന്മാരും കൊലപാതകികൾക്കും ഒക്കെ ഭിമിയിൽ നല്ല ആയുസ്സാണ് .. നല്ലവരായ ആൾക്കാർ രോഗങ്ങൾ വന്നും ആക്\u200cസിഡന്റ് ലും മരിക്കുന്നു',
 'പാവം എത്ര വേദന സഹിച്ചു',
 'ലാലേട്ടന്റെ പെങ്ങളായി അഭിനയിച്ചു കണ്ടു കൊതി തീർന്നില്ല അതിനു മുന്പേ യാത്രയായി .... ആദരാഞ്ജലികൾ',
 'നന്ദുവിനും ശരണ്യ മോൾക്കും',
 'പാവം ആ ചിരിക്ക്\u200c മുന്നിൽ തോറ്റു പോകും എല്ലാം',
 'തിരിച്ചു വരുമെന്ന് കരുതിയിരുന്നു ആരുടെയും പ്രാർഥന കേട്ടില്ല വേദനയോടെ പ്രണാമം',
 'ആ അമ്മയുടെ അവസ്ഥ എന്തായിരിക്കും',
 'ഇനിയാർക്കൂ ഈശ്വരൻ ഈ വിധി നൽകാതിരിക്കട്ടെ',
 'പാവം ഒത്തിരി ഇഷ്ടമായിരുന്നു.സുന്ദരി കുട്ട്യേ',
 'ചേർന്ന് നിൽക്കേണ്ടാവർ പിൻതിരിഞ്ഞ് പോകുമ്പോൾ വേദനയെകൾ ഭയാനകമാണ് ആ നഷ്ടം',
 'ദുനിയാവിൽ ഒന്നും നിലനിൽക്കില്ല

<b>Implementing BERT

In [ ]:
num_emotions = df['Emotion'].nunique()
num_emotions

3

In [ ]:
# Load BERT tokenizer and model for sequence classification
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_emotions)


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize and encode the text data
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
train_encodings

{'input_ids': [[101, 1321, 111398, 15035, 25344, 75984, 34590, 17896, 93330, 72643, 13622, 111395, 65125, 1357, 27921, 111400, 18242, 1355, 61725, 1352, 15035, 18188, 40799, 13622, 1353, 65125, 1364, 29400, 111393, 25344, 25715, 1348, 15035, 30999, 1364, 29400, 111393, 25344, 1323, 37591, 24458, 59956, 1361, 27921, 17896, 59956, 78762, 1353, 27921, 75984, 1364, 29400, 111393, 25344, 18395, 101342, 1352, 37591, 1357, 25344, 17014, 108628, 21925, 1325, 42870, 47025, 67691, 1322, 111393, 23290, 24458, 86678, 111389, 55007, 19078, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1321, 36877, 40979, 106699, 18384, 1348, 18384, 42870, 111385, 15035, 18395, 89291, 111395, 27921, 1322, 1321, 47025, 104612, 1328, 50061, 59956, 27921, 62778, 1367, 43877, 20854, 27921, 39578, 1325, 36089, 24164, 42870, 51454, 1322, 1357, 18384, 111382, 17676, 78833, 14885, 55328, 1367, 69718, 17896, 53144, 1348, 24458, 23290, 72966, 41950, 27921,

In [ ]:
# Convert labels to PyTorch tensors
y_train = torch.tensor(y_train.tolist())
y_test = torch.tensor(y_test.tolist())

In [ ]:
# Create DataLoader for training and test sets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    y_train
)
test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    y_test
)

In [ ]:
train_encodings['input_ids']

[[101,
  1321,
  111398,
  15035,
  25344,
  75984,
  34590,
  17896,
  93330,
  72643,
  13622,
  111395,
  65125,
  1357,
  27921,
  111400,
  18242,
  1355,
  61725,
  1352,
  15035,
  18188,
  40799,
  13622,
  1353,
  65125,
  1364,
  29400,
  111393,
  25344,
  25715,
  1348,
  15035,
  30999,
  1364,
  29400,
  111393,
  25344,
  1323,
  37591,
  24458,
  59956,
  1361,
  27921,
  17896,
  59956,
  78762,
  1353,
  27921,
  75984,
  1364,
  29400,
  111393,
  25344,
  18395,
  101342,
  1352,
  37591,
  1357,
  25344,
  17014,
  108628,
  21925,
  1325,
  42870,
  47025,
  67691,
  1322,
  111393,
  23290,
  24458,
  86678,
  111389,
  55007,
  19078,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [101,
  1321,
  36877,
  40979,
  106699,
  18384,
  1348,
  18384,
  42870,
  111385,
  15035,
  18395,
  89291,
  111395,
  27921,
  1322,
  1321,


In [ ]:
train_encodings['attention_mask']

[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [ ]:
# Define batch size and create DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

data loaders are crucial for efficient training and testing of your model, as they handle the process of loading batches of data, shuffling, and managing the data feeding process during training and testing loops.

In [ ]:
# Define optimizer and training parameters
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 3

In [ ]:
y_train

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
        2, 2, 1, 2, 2, 2, 2, 2])

In [ ]:
tokenizer.decode(train_encodings['input_ids'][7])

'[CLS] പാവം ആ ചിരിക്ക് മുന്നിൽ തോറ്റു പോകും എല്ലാം [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()

In [ ]:
# Validation loop
model.eval()

all_preds = []
all_labels = []

In [ ]:
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

In [ ]:
comment = "ഇദ് കാണുമ്പോൾ നെഞ്ച് പിടയുന്നു"
encoding = tokenizer(comment, truncation=True, padding=True, return_tensors="pt")

with torch.no_grad():
    model.eval()
    outputs = model(**encoding)
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
predicted_label

2

<b>Evaluating The Model

In [ ]:
# Evaluate the model
accuracy = accuracy_score(all_labels, all_preds)
classification_rep = classification_report(all_labels, all_preds)
print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.8666666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         1
           2       0.87      1.00      0.93        13

    accuracy                           0.87        15
   macro avg       0.29      0.33      0.31        15
weighted avg       0.75      0.87      0.80        15



In [ ]:
##